# Build AML Pipeline with azureml modules

In this tutorial you will learn how to work with Azure ML module:

1. Setup enrivonment - install module CLI and module/pipeline SDK
2. Register a few sample modules into your aml workspace using CLI
3. Use module/pipeline SDK to create a pipeline with modules registered in step 2

## Prerequisite
* Install Azure CLI, please follow [the Azure CLI installation instructions](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) to install.

## Setup environment
* Install Azure CLI AML extension which includes the _module_ command group
* Install Azure ML SDK including the APIs to work with _module_ and _pipeline_

In [ ]:
# Uninstall azure-cli-ml (the `az ml` commands)
!az extension remove -n azure-cli-ml 

# Install local version of azure-cli-ml (which includes `az ml module` commands)
!az extension add --source https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/13766063/azure_cli_ml-0.1.0.13766063-py3-none-any.whl --pip-extra-index-urls https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/13766063 --yes 

In [ ]:
# Verify the availability of `az ml module` commands
!az ml module -h

In [ ]:
# Install azureml-sdk with Pipeline, Module
# Important! After install succeed, need to restart kernel

%config IPCompleter.greedyaz=True 
!pip install azureml-pipeline-wrapper==0.1.0.14253903 --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/14253903 --user --upgrade

## Register AML Module

You can manage AML module through [azure-cli-ml](https://aka.ms/moduledoc) or [ml.azure.com](https://ml.azure.com/). <br>

Module could be registered from:
- local path
- public Github url
- Azure DevOps build artifacts

Azureml module support multiple module type:
- Basic python module
- Mpi module
- Parallel run module
- Hdi module (pending on backend support)

In [1]:
# you need to configure your ws information here

subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
workspace_name = 'lisal-amlservice'
resource_group = 'lisal-dev'

# Specify available aml compute in workspace
pipeline_compute = "always-on-ds2v2"

In [ ]:
# Configure your aml workspace 

!az login 
!az account set -s $subscription_id 
!az ml folder attach -w $workspace_name -g $resource_group 

In [ ]:
!az ml module register --spec-file=https://github.com/microsoft/recommenders/blob/master/reco_utils/azureml/azureml_designer_modules/module_specs/sar_train.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/microsoft/recommenders/blob/master/reco_utils/azureml/azureml_designer_modules/module_specs/stratified_splitter.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/microsoft/recommenders/blob/master/reco_utils/azureml/azureml_designer_modules/module_specs/sar_score.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/microsoft/recommenders/blob/master/reco_utils/azureml/azureml_designer_modules/module_specs/recall_at_k.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/microsoft/recommenders/blob/master/reco_utils/azureml/azureml_designer_modules/module_specs/map.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/microsoft/recommenders/blob/master/reco_utils/azureml/azureml_designer_modules/module_specs/ndcg.yaml --set-as-default-version
!az ml module register --spec-file=https://github.com/microsoft/recommenders/blob/master/reco_utils/azureml/azureml_designer_modules/module_specs/precision_at_k.yaml --set-as-default-version

## Create pipeline
You can build pipeline through SDK experience, or drag-n-drop way through [Designer](https://ml.azure.com/visualinterface?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1&flight=cm,nml,newGraphDetail,newGraphAuthoring,all&tid=72f988bf-86f1-41af-91ab-2d7cd011db47) in workspace portal

The new SDK:
* Symplified the syntax to provide consistent experience with drag-n-drop
* Support intellisense and docstring, free you to work with dict all the time
* Support creating a pipeline with unpublished module

In [2]:
from azureml.core import Workspace, Run, Dataset, Datastore
from azureml.pipeline.wrapper import Pipeline, Module, dsl

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

# get modules
sar_split = Module.load(ws, namespace='microsoft.com/cat', name='Stratified Splitter')
sar_train = Module.load(ws, namespace='microsoft.com/cat', name='SAR Training')
sar_score = Module.load(ws, namespace='microsoft.com/cat', name='SAR Scoring')
sar_map = Module.load(ws, namespace='microsoft.com/cat', name='MAP')
sar_ndcg = Module.load(ws, namespace='microsoft.com/cat', name='nDCG')
sar_recall = Module.load(ws, namespace='microsoft.com/cat', name='Recall at K')
sar_precision = Module.load(ws, namespace='microsoft.com/cat', name='Precision at K')

# get dataset
data_name = 'Movie_Ratings'

if data_name not in ws.datasets:
    global_datastore = Datastore(ws, name="azureml_globaldatasets")
    movie_ratings_data = Dataset.File.from_files(global_datastore.path('GenericCSV/Movie_Ratings')).register(workspace=ws, name='Movie_Ratings', description='movie rating data')
    print('Registerd')

movie_ratings_data = ws.datasets[data_name]
print('Training dataset found in workspace')

Training dataset found in workspace


In [3]:
#define a sub pipeline
@dsl.pipeline(name = 'recommender-evaluation', 
              description = 'recall, precision, ndcg, map')
def sar_eval(user_col, item_col, rating_col, precision_col, method, top_n, threshold, test_input, score_input):   
    recall = sar_recall(user_column=user_col, item_column=item_col, rating_column=rating_col, prediction_column=precision_col,
                       relevancy_method=method, top_k=top_n, threshold=threshold,
                       rating_true=test_input,
                       rating_pred=score_input)
    
    map = sar_map(user_column=user_col, item_column=item_col, rating_column=rating_col, prediction_column=precision_col,
                 relevancy_method=method, top_k=top_n, threshold=threshold,
                 rating_true=test_input,
                 rating_pred=score_input)
    
    precision = sar_precision(user_column=user_col, item_column=item_col, rating_column=rating_col, prediction_column=precision_col,
                 relevancy_method=method, top_k=top_n, threshold=threshold,
                 rating_true=test_input,
                 rating_pred=score_input)
    
    ndcg = sar_ndcg(user_column=user_col, item_column=item_col, rating_column=rating_col, prediction_column=precision_col,
                 relevancy_method=method, top_k=top_n, threshold=threshold,
                 rating_true=test_input,
                 rating_pred=score_input)    
    
    return {'recall': recall.outputs.score, 'precision': precision.outputs.score, 'ndcg': ndcg.outputs.score, 'map': map.outputs.score}

In [4]:
#define a pipeline
@dsl.pipeline(name = 'recommender pipeline', 
              description = 'recommender pipeline with SAR algo',
              default_compute_target = pipeline_compute)
def sar_pipeline(top_n, threshold):
    split = sar_split(input_path=movie_ratings_data, 
                      ratio=0.75, seed=42, 
                      user_column='UserId', item_column='MovieId')
    
    train = sar_train(input_path=split.outputs.output_train_data, 
                      user_column='UserId', item_column='MovieId', rating_column='Rating', timestamp_column='Timestamp',
                      time_decay=False, normalize=False)
    
    
    score = sar_score(trained_model=train.outputs.output_model, dataset_to_score=split.outputs.output_test_data,
                     score_type='Item recommendation', ranking_metric='Rating', top_k=top_n, sort_top_k=True)
    
    eval = sar_eval('UserId', 'MovieId', 'Rating', 'prediction', 
                    'top_k', 10, 1.0, 
                    split.outputs.output_test_data, score.outputs.score_result)
    
    return {**eval.outputs}

pipeline = sar_pipeline(10, 1.0)

In [5]:
# validate pipeline and visualize the graph
pipeline.validate()

{'result': 'validation passed', 'errors': []}

In [6]:
# save as a draft
pipeline.save(experiment_name = 'recommender-pipeline-sar', id = '767f99ef-9d0d-4518-b9aa-9faf0170b954')

Name,Id,Tags,Properties,Last Submitted Pipeline Run Id
recommender pipeline,767f99ef-9d0d-4518-b9aa-9faf0170b954,,,


In [7]:
run = pipeline.submit(experiment_name = 'recommender-pipeline-sar')
run.wait_for_completion()

Submitted PipelineRun a4e04997-8dc1-4b49-85ea-d814dcc12dc9
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/recommender-pipeline-sar/runs/a4e04997-8dc1-4b49-85ea-d814dcc12dc9?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/lisal-dev/workspaces/lisal-amlservice
PipelineRunId: a4e04997-8dc1-4b49-85ea-d814dcc12dc9
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/recommender-pipeline-sar/runs/a4e04997-8dc1-4b49-85ea-d814dcc12dc9?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/lisal-dev/workspaces/lisal-amlservice
PipelineRun Status: NotStarted
